# Week 3 Assignment:  Componentes de la canalización de datos para el ML de producción

En este último ejercicio de programación calificado del curso, juntarás todas las lecciones que hemos cubierto hasta ahora para manejar los tres primeros pasos de un proyecto de machine learning de producción - ingestión de datos, validación de datos y transformación de datos.

Específicamente, usted construirá la tubería de datos de producción por:

* Realizar la selección de características
* Ingesta del conjunto de datos
* Generar las estadísticas del conjunto de datos
* Creación de un esquema según el conocimiento del dominio
* Creación de entornos de esquema
* Visualización de las anomalías del conjunto de datos
* Preprocesamiento, transformación e ingeniería de sus características
* Seguimiento de la procedencia de su canalización de datos utilizando metadatos ML

La mayoría de estos temas ya le resultarán familiares, así que intente hacer los ejercicios recordando o consultando la documentación. Sin embargo, si te quedas atascado, puedes repasar las lecciones en clase y los laboratorios no calificados. 

¡Comencemos!

## Table of Contents

- [1 - Imports](#1)
- [2 - Load the Dataset](#2)
- [3 - Feature Selection](#4)
  - [Exercise 1 - Feature Selection](#ex-1)
- [4 - Data Pipeline](#4)
  - [4.1 - Setup the Interactive Context](#4-1)
  - [4.2 - Generating Examples](#4-2)
    - [Exercise 2 - ExampleGen](#ex-2)
  - [4.3 - Computing Statistics](#4-3)
    - [Exercise 3 - StatisticsGen](#ex-3)
  - [4.4 - Inferring the Schema](#4-4)
    - [Exercise 4 - SchemaGen](#ex-4)
  - [4.5 - Curating the Schema](#4-5)
    - [Exercise 5 - Curating the Schema](#ex-5)
  - [4.6 - Schema Environments](#4-6)
    - [Exercise 6 - Define the serving environment](#ex-6)
  - [4.7 - Generate new statistics using the updated schema](#4-7)
      - [Exercise 7 - ImportSchemaGen](#ex-7)
      - [Exercise 8 - StatisticsGen with the new schema](#ex-8)
  - [4.8 - Check anomalies](#4-8)
      - [Exercise 9 - ExampleValidator](#ex-9)
  - [4.9 - Feature Engineering](#4-9)
      - [Exercise 10 - preprocessing function](#ex-10)
      - [Exercise 11 - Transform](#ex-11)
- [5 - ML Metadata](#5)
  - [5.1 - Accessing stored artifacts](#5-1)
  - [5.2 - Tracking artifacts](#5-2)
    - [Exercise 12 - Get parent artifacts](#ex-12)

In [1]:
import tensorflow as tf
from tfx import v1 as tfx

# TFX libraries
import tensorflow_data_validation as tfdv
import tensorflow_transform as tft
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

# For performing feature selection
from sklearn.feature_selection import SelectKBest, f_classif

# For feature visualization
import matplotlib.pyplot as plt 
import seaborn as sns

# Utilities
from tensorflow.python.lib.io import file_io
from tensorflow_metadata.proto.v0 import schema_pb2
from google.protobuf.json_format import MessageToDict
from  tfx.proto import example_gen_pb2
from tfx.types import standard_artifacts
from tensorflow_transform.tf_metadata import dataset_metadata, schema_utils
import tensorflow_transform.beam as tft_beam
import os
import pprint
import tempfile
import pandas as pd

# To ignore warnings from TF
tf.get_logger().setLevel('ERROR')

# For formatting print statements
pp = pprint.PrettyPrinter()

# Display versions of TF and TFX related packages
print('TensorFlow version: {}'.format(tf.__version__))
print('TFX version: {}'.format(tfx.__version__))
print('TensorFlow Data Validation version: {}'.format(tfdv.__version__))
print('TensorFlow Transform version: {}'.format(tft.__version__))

TensorFlow version: 2.6.0
TFX version: 1.3.0
TensorFlow Data Validation version: 1.3.0
TensorFlow Transform version: 1.3.0


<a name='2'></a>
## 2 - Cargar el conjunto de datos

Va a utilizar una variante del conjunto de datos [Cover Type](https://archive.ics.uci.edu/ml/datasets/covertype). Este puede ser usado para entrenar un modelo que predice el tipo de cobertura forestal basado en variables cartográficas. Puede leer más sobre el conjunto de datos *original* [aquí](https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.info) y hemos resumido las columnas de datos a continuación:

| Column Name | Variable Type | Units / Range | Description |
| --------- | ------------ | ----- | ------------------- |
| Elevation | quantitative |meters | Elevation in meters |
| Aspect | quantitative | azimuth | Aspect in degrees azimuth |
| Slope | quantitative | degrees | Slope in degrees |
| Horizontal_Distance_To_Hydrology | quantitative | meters | Horz Dist to nearest surface water features |
| Vertical_Distance_To_Hydrology | quantitative | meters | Vert Dist to nearest surface water features |
| Horizontal_Distance_To_Roadways | quantitative | meters | Horz Dist to nearest roadway |
| Hillshade_9am | quantitative | 0 to 255 index | Hillshade index at 9am, summer solstice |
| Hillshade_Noon | quantitative | 0 to 255 index | Hillshade index at noon, summer soltice |
| Hillshade_3pm | quantitative | 0 to 255 index | Hillshade index at 3pm, summer solstice |
| Horizontal_Distance_To_Fire_Points | quantitative | meters | Horz Dist to nearest wildfire ignition points |
| Wilderness_Area (4 binary columns) | qualitative | 0 (absence) or 1 (presence) | Wilderness area designation |
| Soil_Type (40 binary columns) | qualitative | 0 (absence) or 1 (presence) | Soil Type designation |
| Cover_Type (7 types) | integer | 1 to 7 | Forest Cover Type designation |

Como puede observar, los datos cualitativos ya han sido codificados en un solo paso (por ejemplo, `Soil_Type` "Tipo de suelo" tiene 40 columnas binarias en las que un "1" indica la presencia de una característica). Para el aprendizaje, utilizaremos una versión modificada de este conjunto de datos que muestra un formato más crudo. Esto le permitirá practicar sus habilidades en el manejo de diferentes tipos de datos. Puede ver el código para preparar el conjunto de datos [aquí](https://github.com/GoogleCloudPlatform/mlops-on-gcp/blob/master/datasets/covertype/wrangle/prepare.ipynb) si lo desea, pero no es necesario para esta tarea**. Los principales cambios incluyen:

* Convertir el `Wilderness_Area` "Área_Salvaje" y el `Soil_Type` "Tipo de suelo"  en cadenas.
* Convertir el rango de `Cover_Type` "Tipo_de_cubierta" a [0, 6]

Ejecute las siguientes celdas para cargar el conjunto de datos **modificado** en su espacio de trabajo. 

In [2]:
# OPCIONAL: En caso de que quieras reiniciar el espacio de trabajo del laboratorio *desde cero*, puedes
# puede descomentar y ejecutar este bloque para borrar los archivos y directorios
# directorios creados anteriormente. 

# !rm -rf pipeline
# !rm -rf data

In [2]:
# Declarar las rutas de acceso a los datos
DATA_DIR = './data'
TRAINING_DIR = f'{DATA_DIR}/training'
TRAINING_DATA = f'{TRAINING_DIR}/dataset.csv'

# Create the directory
!mkdir -p {TRAINING_DIR}

In [3]:
# download the dataset
!wget -nc https://storage.googleapis.com/mlep-public/course_2/week3/dataset.csv -P {TRAINING_DIR}

File ‘./data/training/dataset.csv’ already there; not retrieving.



<a name='3'></a>
## 3 - Selección de características

Para su primera tarea, reducirá el número de características que alimentará al modelo. Como se mencionó en la semana 2, esto ayudará a reducir la complejidad de su modelo y ahorrar recursos durante el entrenamiento. Supongamos que ya tiene un modelo de referencia entrenado con todas las características y quiere ver si la reducción del número de características genera un modelo mejor. Querrá seleccionar un subconjunto que tenga un gran valor predictivo para la etiqueta (en este caso el `Tipo de cubierta`). Hagamos eso en las siguientes celdas.

In [4]:
# Load the dataset to a dataframe
df = pd.read_csv(TRAINING_DATA)

# Preview the dataset
df.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,Rawah,C7745,4
1,2590,56,2,212,-6,390,220,235,151,6225,Rawah,C7745,4
2,2804,139,9,268,65,3180,234,238,135,6121,Rawah,C4744,1
3,2785,155,18,242,118,3090,238,238,122,6211,Rawah,C7746,1
4,2595,45,2,153,-1,391,220,234,150,6172,Rawah,C7745,4


In [5]:
# Show the data type of each column
df.dtypes

Elevation                              int64
Aspect                                 int64
Slope                                  int64
Horizontal_Distance_To_Hydrology       int64
Vertical_Distance_To_Hydrology         int64
Horizontal_Distance_To_Roadways        int64
Hillshade_9am                          int64
Hillshade_Noon                         int64
Hillshade_3pm                          int64
Horizontal_Distance_To_Fire_Points     int64
Wilderness_Area                       object
Soil_Type                             object
Cover_Type                             int64
dtype: object

In [6]:
# copia del dataset
df_num = df.copy()

# Columnas categoricas
cat_columns = ['Wilderness_Area', 'Soil_Type']

# Etiquetas
label_column = ['Cover_Type']

# eliminamos columnas categoricas y las etiquetas
df_num.drop(cat_columns, axis=1, inplace=True)
df_num.drop(label_column, axis=1, inplace=True)

# Preview the resuls
df_num.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points
0,2596,51,3,258,0,510,221,232,148,6279
1,2590,56,2,212,-6,390,220,235,151,6225
2,2804,139,9,268,65,3180,234,238,135,6121
3,2785,155,18,242,118,3090,238,238,122,6211
4,2595,45,2,153,-1,391,220,234,150,6172


Utilizará los módulos incorporados de scikit-learn para realizar la [selección de características univariantes](https://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection) en los atributos numéricos de nuestro conjunto de datos. En primer lugar, es necesario preparar las características de entrada y de destino:

In [7]:
# Etiquetas 
y = df[label_column].values

# Caracteristicas
X = df_num.values

In [8]:
X,y

(array([[2596,   51,    3, ...,  232,  148, 6279],
        [2590,   56,    2, ...,  235,  151, 6225],
        [2804,  139,    9, ...,  238,  135, 6121],
        ...,
        [2386,  159,   17, ...,  241,  130,  854],
        [2384,  170,   15, ...,  245,  143,  864],
        [2383,  165,   13, ...,  244,  141,  875]]),
 array([[4],
        [4],
        [1],
        ...,
        [2],
        [2],
        [2]]))

Después, utilizará [SelectKBest](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html#sklearn.feature_selection.SelectKBest) para puntuar cada característica de entrada frente a la variable objetivo. Tenga en cuenta la función de puntuación que va a pasar y asegúrese de que es apropiada para los valores de entrada (numéricos) y de destino (categóricos).

<a name='ex-1'></a>
### Ejercicio 1: Selección de características

Completa el código siguiente para seleccionar las 8 primeras características de las columnas numéricas.

In [9]:
### START CODE HERE ###

# Crear el objeto SelectKBest utilizando f_classif (estadística ANOVA) para 8 clases
select_k_best = SelectKBest(score_func = f_classif, k = 8)

# Ajustar y transformar los datos de entrada mediante select_k_best
X_new = select_k_best.fit_transform(X,y)

# Extraer las características seleccionadas mediante la API get_support
features_mask = select_k_best.get_support()

### END CODE HERE ###

# Print the results
reqd_cols = pd.DataFrame({'Columns': df_num.columns, 'Retain': features_mask})
print(reqd_cols)

                              Columns  Retain
0                           Elevation    True
1                              Aspect   False
2                               Slope    True
3    Horizontal_Distance_To_Hydrology    True
4      Vertical_Distance_To_Hydrology    True
5     Horizontal_Distance_To_Roadways    True
6                       Hillshade_9am    True
7                      Hillshade_Noon    True
8                       Hillshade_3pm   False
9  Horizontal_Distance_To_Fire_Points    True


**Expected Output:**

```
                              Columns  Retain
0                           Elevation    True
1                              Aspect   False
2                               Slope    True
3    Horizontal_Distance_To_Hydrology    True
4      Vertical_Distance_To_Hydrology    True
5     Horizontal_Distance_To_Roadways    True
6                       Hillshade_9am    True
7                      Hillshade_Noon    True
8                       Hillshade_3pm   False
9  Horizontal_Distance_To_Fire_Points    True
```

Si ha obtenido los resultados esperados, ahora puede seleccionar este subconjunto de características del marco de datos original y guardarlo en un nuevo directorio de su espacio de trabajo.

In [10]:
# Establecer las rutas de acceso al conjunto de datos reducido
TRAINING_DIR_FSELECT = f'{TRAINING_DIR}/fselect'
TRAINING_DATA_FSELECT = f'{TRAINING_DIR_FSELECT}/dataset.csv'

# Create the directory
!mkdir -p {TRAINING_DIR_FSELECT}

In [11]:
# Obtener los nombres de las características de SelectKBest
feature_names = list(df_num.columns[features_mask])

# Añadir las columnas de categorías y etiquetas
feature_names = feature_names + cat_columns + label_column

# Seleccionar el subconjunto de columnas seleccionado
df_select = df[feature_names]

# Escribir el CSV en el directorio creado
df_select.to_csv(TRAINING_DATA_FSELECT, index=False)

# Previsualizar los resultados
df_select.head()

,Elevation,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2596,3,258,0,510,221,232,6279,Rawah,C7745,4
1,2590,2,212,-6,390,220,235,6225,Rawah,C7745,4
2,2804,9,268,65,3180,234,238,6121,Rawah,C4744,1
3,2785,18,242,118,3090,238,238,6211,Rawah,C7746,1
4,2595,2,153,-1,391,220,234,6172,Rawah,C7745,4


<a name='4'></a>
## 4 - Canalización de datos

Con el subconjunto de características seleccionado preparado, ahora puede comenzar a construir la tubería de datos. Esto implica ingerir, validar y transformar sus datos. Utilizarás los componentes de TFX que ya has encontrado en los laboratorios no graduados y que puedes consultar aquí en la [documentación oficial](https://www.tensorflow.org/tfx/api_docs/python/tfx/components).

<a name='4-1'></a>
### 4.1 - Configurar el Contexto Interactivo

Como es habitual, primero configurará el Contexto Interactivo para poder ejecutar manualmente los componentes del pipeline desde el cuaderno. Guardará la base de datos sqlite en un directorio predefinido en su espacio de trabajo. Por favor, no modifique esta ruta porque la necesitará en un ejercicio posterior relacionado con los Metadatos ML.

In [12]:
# Location of the pipeline metadata store
PIPELINE_DIR = './pipeline'

# Declare the InteractiveContext and use a local sqlite file as the metadata store.
context = InteractiveContext(pipeline_root=PIPELINE_DIR)

<a name='4-2'></a>
### 4.2 - Generación de ejemplos

El primer paso en el pipeline es ingerir los datos. Utilizando [ExampleGen](https://www.tensorflow.org/tfx/guide/examplegen), puede convertir los datos en bruto en TFRecords para un cálculo más rápido en las etapas posteriores del pipeline.

<a name='ex-2'></a>
#### Ejercicio 2: ExampleGen

Utiliza `ExampleGen` para ingerir el conjunto de datos que cargamos anteriormente. Algunas cosas a tener en cuenta:

* La entrada está en formato CSV, así que necesitarás usar el tipo apropiado de `ExampleGen` para manejarlo. 
* Esta función acepta una ruta de *directorio* a los datos de entrenamiento y no la ruta del archivo CSV. 

Esta función tardará un par de minutos en ejecutarse.

In [14]:
# NOTA: Descomente y ejecute esto si obtiene un error diciendo que hay diferentes 
# cabeceras en el conjunto de datos. Esto suele ser debido a los puntos de control del cuaderno guardados en 
# esa carpeta.
# !rm -rf {TRAINING_DIR}/.ipynb_checkpoints
# !rm -rf {TRAINING_DIR_FSELECT}/.ipynb_checkpoints
# !rm -rf {SERVING_DIR}/.ipynb_checkpoints

In [13]:
### START CODE HERE

# Instanciar ExampleGen con el conjunto de datos CSV de entrada
example_gen =  tfx.components.CsvExampleGen(input_base=TRAINING_DIR_FSELECT)

# Ejecutar el componente utilizando la instancia InteractiveContext
context.run(example_gen)

### END CODE HERE

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 27
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 22
        type_id: 14
        uri: "./pipeline/CsvExampleGen/examples/27"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "file_format"
          value {
            string_value: "tfrecords_gzip"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:27713036,xor_checksum:1668797881,sum_checksum:1668797881"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

<a name='4-3'></a>
### 4.3 - Cálculo de estadísticas

A continuación, calculará las estadísticas de sus datos. Esto le permitirá observar y analizar las características de sus datos a través de las visualizaciones proporcionadas por la biblioteca integrada [FACETS](https://pair-code.github.io/facets/).

<a name='ex-3'></a>
#### Ejercicio 3: StatisticsGen

Utilice [StatisticsGen](https://www.tensorflow.org/tfx/guide/statsgen) para calcular las estadísticas de los ejemplos de salida de `ExampleGen`. 

In [14]:
### START CODE HERE

# Instantiate StatisticsGen with the ExampleGen ingested dataset
statistics_gen = tfx.components.StatisticsGen(examples = example_gen.outputs["examples"])
    

# Run the component
context.run(statistics_gen)
### END CODE HERE

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 28
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 23
        type_id: 16
        uri: "./pipeline/StatisticsGen/statistics/28"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 16
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [37]:
# Display the results
context.show(statistics_gen.outputs['statistics'])

Una vez que haya cargado la visualización, podrá observar que la columna `zeros` para `Cover_type` está resaltada en rojo. La visualización nos está avisando de que esto podría ser un problema potencial. En nuestro caso, sin embargo, sabemos que el `Tipo de cubierta` tiene un rango de [0, 6], por lo que tener ceros en esta columna es algo que esperamos.

<a name='4-4'></a>
### 4.4 - Inferir el esquema

Necesitará crear un esquema para validar los conjuntos de datos entrantes durante el entrenamiento y el servicio. Afortunadamente, TFX le permite inferir un primer borrador de este esquema con el componente [SchemaGen](https://www.tensorflow.org/tfx/guide/schemagen).

<a name='ex-4'></a>
#### Exercise 4: SchemaGen

Use `SchemaGen` to infer a schema based on the computed statistics of `StatisticsGen`

In [17]:
### START CODE HERE
# Instantiate SchemaGen with the output statistics from the StatisticsGen
schema_gen = tfx.components.SchemaGen(statistics=statistics_gen.outputs["statistics"])
    
    

# Run the component
context.run(schema_gen)
### END CODE HERE

ExecutionResult(
    component_id: SchemaGen
    execution_id: 29
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 24
        type_id: 18
        uri: "./pipeline/SchemaGen/schema/29"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 18
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [18]:
# Visualize the output
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,,-
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,-
'Hillshade_Noon',INT,required,,-
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,-
'Soil_Type',STRING,required,,'Soil_Type'


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C5151', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


<a name='4-5'></a>
### 4.5 - Curar el esquema

Puede ver que el esquema inferido es capaz de capturar los tipos de datos correctamente y también es capaz de mostrar los valores esperados para los datos cualitativos (es decir, cadena). Sin embargo, todavía se puede afinar. Por ejemplo, tenemos características en las que esperamos un determinado rango:

* `Hillshade_9am`: 0 to 255
* `Hillshade_Noon`: 0 to 255
* `Slope`: 0 to 90
* `Cover_Type`:  0 to 6


Es conveniente que actualices tu esquema para que tome nota de estos valores, de forma que el pipeline pueda detectar si se están introduciendo valores no válidos en el modelo.

<a name='ex-5'></a>
#### Ejercicio 5: Curar el esquema

Utilice [TFDV](https://www.tensorflow.org/tfx/data_validation/get_started) para actualizar el esquema inferido para restringir un rango de valores a las características mencionadas anteriormente.

Cosas a tener en cuenta:
* Puede utilizar [tfdv.set_domain()](https://www.tensorflow.org/tfx/data_validation/api_docs/python/tfdv/set_domain) para definir los valores aceptables para una característica concreta.
* Estos deben seguir siendo tipos INT después de hacer sus cambios.
* Declare `Cover_Type` como una variable *categórica*. A diferencia de las otras cuatro características, los enteros de 0 a 6 corresponden aquí a una etiqueta designada y no a una medida cuantitativa. Puedes mirar las banderas disponibles para `set_domain()` en la doc. oficial para saber cómo configurar esto.

In [19]:
try:
    # Get the schema uri
    schema_uri = schema_gen.outputs['schema']._artifacts[0].uri
    
# para la graduación ya que context.run() no funciona fuera del cuaderno
except IndexError:
    print("context.run() was no-op")
    schema_path = './pipeline/SchemaGen/schema'
    dir_id = os.listdir(schema_path)[0]
    schema_uri = f'{schema_path}/{dir_id}'

In [20]:
# Get the schema pbtxt file from the SchemaGen output
schema = tfdv.load_schema_text(os.path.join(schema_uri, 'schema.pbtxt'))

In [22]:
### START CODE HERE ###
# Configure las dos características `Hillshade` para que tengan un rango de 0 a 255
# Set the two `Hillshade` features to have a range of 0 to 255
tfdv.set_domain(schema, 'Hillshade_9am', schema_pb2.IntDomain(
    name = 'Hillshade_9am', 
    min = 0, max = 255))
tfdv.set_domain(schema, 'Hillshade_Noon', schema_pb2.IntDomain(
    name = 'Hillshade_Noon', 
    min=0, max=255))

# Set the `Slope` feature to have a range of 0 to 90
# Establecer la característica `Slope` para tener un rango de 0 a 90
tfdv.set_domain(schema, 'Slope', schema_pb2.IntDomain(name='Slope', min=0, max=90))

# Set `Cover_Type` to categorical having minimum value of 0 and maximum value of 6
# Establecer `Cover_Type` como categórico teniendo un valor mínimo de 0 y un valor máximo de 6
tfdv.set_domain(schema, 'Cover_Type', schema_pb2.IntDomain(
                        name  = 'Cover_Type', 
                        min   = 0, 
                        max   = 6, 
                        is_categorical = True))

### END CODE HERE ###

tfdv.display_schema(schema=schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Soil_Type',STRING,required,,'Soil_Type'
'Wilderness_Area',STRING,required,,'Wilderness_Area'
'Cover_Type',INT,required,,min: 0; max: 6
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,min: 0; max: 255
'Hillshade_Noon',INT,required,,min: 0; max: 255
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C5151', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


<a name='4-6'></a>
### 4.6 - Entornos de esquemas

En el aprendizaje supervisado, entrenamos el modelo para hacer predicciones alimentando un conjunto de características con su correspondiente etiqueta. Así, nuestro conjunto de datos de entrenamiento tendrá tanto las características de entrada como la etiqueta, y el esquema está configurado para detectarlas. 

Sin embargo, después del entrenamiento y de servir el modelo para la inferencia, los datos de entrada ya no tendrán la etiqueta. Esto presentará problemas al validar los datos utilizando la versión actual del esquema. Vamos a demostrarlo en las siguientes celdas. Simulará un conjunto de datos de servicio obteniendo un subconjunto del conjunto de entrenamiento y eliminando la columna de la etiqueta (es decir, `Tipo de cubierta`). A continuación, validaremos este conjunto de datos de servicio utilizando el esquema que hemos creado anteriormente.

In [23]:
# Declare paths to the serving data
SERVING_DIR = f'{DATA_DIR}/serving'
SERVING_DATA = f'{SERVING_DIR}/serving_dataset.csv'

# Create the directory
!mkdir -p {SERVING_DIR}

In [24]:
# Read a subset of the training dataset
serving_data = pd.read_csv(TRAINING_DATA, nrows=100)

# Drop the `Cover_Type` column
serving_data.drop(columns='Cover_Type', inplace=True)

# Save the modified dataset
serving_data.to_csv(SERVING_DATA, index=False)

# Delete unneeded variable from memory
del serving_data

In [25]:
# Declare StatsOptions to use the curated schema
stats_options = tfdv.StatsOptions(schema=schema, infer_type_from_schema=True)

# Compute the statistics of the serving dataset
serving_stats = tfdv.generate_statistics_from_csv(SERVING_DATA, stats_options=stats_options)

# Detect anomalies in the serving dataset
anomalies = tfdv.validate_statistics(serving_stats, schema=schema)

# Display the anomalies detected
tfdv.display_anomalies(anomalies)

,Anomaly short description,Anomaly long description
Feature name,,
'Cover_Type',Column dropped,Column is completely missing


Como era de esperar, se marca la columna que falta. Para solucionarlo, es necesario configurar el esquema para que detecte cuándo se está utilizando para el entrenamiento o para la inferencia/servicio. Puede hacerlo configurando [entornos de esquema](https://www.tensorflow.org/tfx/tutorials/data_validation/tfdv_basic#schema_environments).

<a name='ex-6'></a>
#### Exercise 6: Define the serving environment

Complete the code below to ignore the `Cover_Type` feature when validating in the *SERVING* environment.

In [26]:
schema.default_environment.append('TRAINING')

# Sugerencia: Crear otro entorno de esquema por defecto con nombre SERVING (pasar una cadena)
schema.default_environment.append("SERVING")

# Eliminar la característica Cover_Type de SERVING usando TFDV
# Sugerencia: pasar las cadenas con el nombre de la característica y el entorno 
tfdv.get_feature(schema, "Cover_Type").not_in_environment.append("SERVING")
### TERMINAR EL CÓDIGO AQUÍ ###

Si se hace correctamente, la ejecución de la celda de abajo debería mostrar *Sin Anomalías*.

In [27]:
# Validate the serving dataset statistics in the `SERVING` environment
anomalies = tfdv.validate_statistics(serving_stats, schema=schema, environment='SERVING')

# Display the anomalies detected
tfdv.display_anomalies(anomalies)

Ahora podemos guardar este esquema curado en un directorio local para poder importarlo a nuestra tubería TFX.

In [28]:
# Declare the path to the updated schema directory
UPDATED_SCHEMA_DIR = f'{PIPELINE_DIR}/updated_schema'

# Create the said directory
!mkdir -p {UPDATED_SCHEMA_DIR}

# Declare the path to the schema file
schema_file = os.path.join(UPDATED_SCHEMA_DIR, 'schema.pbtxt')

# Save the curated schema to the said file
tfdv.write_schema_text(schema, schema_file)

Como comprobación de cordura, mostremos el esquema que acabamos de guardar y verifiquemos que contiene los cambios que hemos introducido. Debería seguir mostrando los rangos en la columna `Domain` y debería haber dos entornos disponibles.

In [29]:
# Load the schema from the directory we just created
new_schema = tfdv.load_schema_text(schema_file)

# Display the schema. Check that the Domain column still contains the ranges.
tfdv.display_schema(schema=new_schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Soil_Type',STRING,required,,'Soil_Type'
'Wilderness_Area',STRING,required,,'Wilderness_Area'
'Cover_Type',INT,required,,min: 0; max: 6
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,min: 0; max: 255
'Hillshade_Noon',INT,required,,min: 0; max: 255
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C5151', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


In [30]:
# The environment list should show `TRAINING` and `SERVING`.
new_schema.default_environment

['TRAINING', 'SERVING']

<a name='4-7'></a>
### 4.7 - Generar nuevas estadísticas utilizando el esquema actualizado

Ahora calculará las estadísticas utilizando el esquema que acaba de curar. Sin embargo, recuerde que los componentes TFX interactúan entre sí obteniendo la información de los artefactos del almacén de metadatos. Así que primero tiene que importar el archivo de esquema curado a ML Metadata. Esto se hará utilizando un [ImportSchemaGen](https://www.tensorflow.org/tfx/api_docs/python/tfx/v1/components/ImportSchemaGen) para crear un artefacto que represente el esquema curado.

<a name='ex-7'></a>
#### Ejercicio 7: ImportSchemaGen

Complete el siguiente código para crear un artefacto `Schema` que apunte a la ruta del archivo de esquema curado.

In [31]:
### START CODE HERE ###

# Use ImportSchemaGen to put the curated schema to ML Metadata
user_schema_importer = tfx.components.ImportSchemaGen( schema_file = schema_file)


# Run the component
context.run(user_schema_importer, enable_cache=False)

### END CODE HERE ###

context.show(user_schema_importer.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Soil_Type',STRING,required,,'Soil_Type'
'Wilderness_Area',STRING,required,,'Wilderness_Area'
'Cover_Type',INT,required,,min: 0; max: 6
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,min: 0; max: 255
'Hillshade_Noon',INT,required,,min: 0; max: 255
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C5151', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


Una vez creado el artefacto con éxito, ahora puedes utilizar `StatisticsGen` y pasar un parámetro `schema` para utilizar el esquema curado.

<a name='ex-8'></a>
#### Exercise 8: Statistics with the new schema

Utiliza `StatisticsGen` para calcular las estadísticas con el esquema que actualizaste en la sección anterior.

In [34]:
### START CODE HERE ###
# Use StatisticsGen to compute the statistics using the curated schema
statistics_gen_updated = tfx.components.StatisticsGen(
                            examples      = example_gen.outputs["examples"],
                            schema        = user_schema_importer.outputs["schema"],
                            stats_options = tfdv.StatsOptions(infer_type_from_schema=True))


# Run the component
context.run(statistics_gen_updated)
### END CODE HERE ###

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 23
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 19
        type_id: 16
        uri: "./pipeline/StatisticsGen/statistics/23"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 16
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [35]:
context.show(user_schema_importer.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Soil_Type',STRING,required,,'Soil_Type'
'Wilderness_Area',STRING,required,,'Wilderness_Area'
'Cover_Type',INT,required,,min: 0; max: 6
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,min: 0; max: 255
'Hillshade_Noon',INT,required,,min: 0; max: 255
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C5151', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


Una vez creado el artefacto con éxito, ahora puedes utilizar `StatisticsGen` y pasar un parámetro `schema` para utilizar el esquema curado.

<a name='ex-8'></a>
#### Exercise 8: Statistics with the new schema

Utiliza `StatisticsGen` para calcular las estadísticas con el esquema que actualizaste en la sección anterior.

In [36]:
### START CODE HERE ###
# Use StatisticsGen to compute the statistics using the curated schema
statistics_gen_updated = tfx.components.StatisticsGen(
                            examples      = example_gen.outputs["examples"],
                            schema        = user_schema_importer.outputs["schema"],
                            stats_options = tfdv.StatsOptions(infer_type_from_schema=True))


# Run the component
context.run(statistics_gen_updated)
### END CODE HERE ###

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 25
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 19
        type_id: 16
        uri: "./pipeline/StatisticsGen/statistics/23"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        create_time_since_epoch: 1667604414531
        last_update_time_since_epoch: 1667604432611
        , artifact_type: id: 16
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [38]:
context.show(statistics_gen_updated.outputs['statistics'])

El gráfico se verá en su mayor parte igual que en las ejecuciones anteriores, pero puede ver que el `Tipo de cobertura` está ahora bajo las características categóricas. Esto demuestra que `StatisticsGen` está utilizando el esquema actualizado.

<a name='4-8'></a>
### 4.8 - Comprobar anomalías

Ahora comprobarás si el conjunto de datos tiene alguna anomalía con respecto al esquema. Puede hacerlo fácilmente con el componente [ExampleValidator](https://www.tensorflow.org/tfx/guide/exampleval).

<a name='ex-9'></a>
#### Ejercicio 9: ExampleValidator

Comprueba si hay alguna anomalía utilizando `ExampleValidator`. Tendrás que pasar las estadísticas y el esquema actualizados de las secciones anteriores.

In [40]:
### START CODE HERE ###

example_validator = tfx.components.ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=user_schema_importer.outputs['schema'])
    
    

# Run the component.
context.run(example_validator)

### END CODE HERE ###
### END CODE HERE ###

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 26
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 21
        type_id: 21
        uri: "./pipeline/ExampleValidator/anomalies/26"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 21
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [41]:
# Visualize the results
context.show(example_validator.outputs['anomalies'])

<a name='4-10'></a>
### 4.10 - Ingeniería de características

Ahora procederá a transformar sus características a una forma adecuada para el entrenamiento de un modelo. Esto puede incluir varios métodos como el escalado y la conversión de cadenas en índices de vocabulario. Es importante que estas transformaciones sean consistentes a través de sus datos de entrenamiento, y también para los datos de servicio cuando el modelo se despliega para la inferencia. TFX asegura esto generando un gráfico que procesará los datos entrantes tanto durante el entrenamiento como durante la inferencia.

Primero declaremos las constantes y la función de utilidad que usaremos para el ejercicio.

In [40]:
# Set the constants module filename
_cover_constants_module_file = 'cover_constants.py'

In [41]:
%%writefile {_cover_constants_module_file}

SCALE_MINMAX_FEATURE_KEYS = [
        "Horizontal_Distance_To_Hydrology",
        "Vertical_Distance_To_Hydrology",
    ]

SCALE_01_FEATURE_KEYS = [
        "Hillshade_9am",
        "Hillshade_Noon",
        "Horizontal_Distance_To_Fire_Points",
    ]

SCALE_Z_FEATURE_KEYS = [
        "Elevation",
        "Slope",
        "Horizontal_Distance_To_Roadways",
    ]

VOCAB_FEATURE_KEYS = ["Wilderness_Area"]

HASH_STRING_FEATURE_KEYS = ["Soil_Type"]

LABEL_KEY = "Cover_Type"

# Utility function for renaming the feature
def transformed_name(key):
    return key + '_xf'

Writing cover_constants.py


A continuación definiremos el `preprocessing_fn` para aplicar transformaciones a las características. 

<a name='ex-10'></a>
#### Exercise 10: Preprocessing function

Complete el módulo para transformar sus características. Consulta los comentarios del código para obtener pistas sobre las operaciones que debes realizar.

Aquí hay algunos enlaces a los docs de las funciones que necesitarás para completar esta función:

- [`tft.scale_by_min_max`](https://www.tensorflow.org/tfx/transform/api_docs/python/tft/scale_by_min_max)
- [`tft.scale_to_0_1`](https://www.tensorflow.org/tfx/transform/api_docs/python/tft/scale_to_0_1)
- [`tft.scale_to_z_score`](https://www.tensorflow.org/tfx/transform/api_docs/python/tft/scale_to_z_score)
- [`tft.compute_and_apply_vocabulary`](https://www.tensorflow.org/tfx/transform/api_docs/python/tft/compute_and_apply_vocabulary)
- [`tft.hash_strings`](https://www.tensorflow.org/tfx/transform/api_docs/python/tft/hash_strings)

In [42]:
# Set the transform module filename
_cover_transform_module_file = 'cover_transform.py'

In [44]:
%%writefile {_cover_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

import cover_constants

_SCALE_MINMAX_FEATURE_KEYS = cover_constants.SCALE_MINMAX_FEATURE_KEYS
_SCALE_01_FEATURE_KEYS = cover_constants.SCALE_01_FEATURE_KEYS
_SCALE_Z_FEATURE_KEYS = cover_constants.SCALE_Z_FEATURE_KEYS
_VOCAB_FEATURE_KEYS = cover_constants.VOCAB_FEATURE_KEYS
_HASH_STRING_FEATURE_KEYS = cover_constants.HASH_STRING_FEATURE_KEYS
_LABEL_KEY = cover_constants.LABEL_KEY
_transformed_name = cover_constants.transformed_name

def preprocessing_fn(inputs):

    features_dict = {}

    ### START CODE HERE ###
    for feature in _SCALE_MINMAX_FEATURE_KEYS:
        data_col = inputs[feature] 
        # Transformar usando el escalado de la función min_max
        # Sugerencia: Utilizar tft.scale_by_min_max pasando la columna correspondiente
        # Utilizar el rango de salida *por defecto* de la función
        features_dict[_transformed_name(feature)] = tft.scale_by_min_max(data_col)

    for feature in _SCALE_01_FEATURE_KEYS:
        data_col = inputs[feature] 
        # Transformar usando la función de escalado de 0 a 1
        # Sugerencia: tft.scale_to_0_1
        features_dict[_transformed_name(feature)] = tft.scale_to_0_1(data_col)

    for feature in _SCALE_Z_FEATURE_KEYS:
        data_col = inputs[feature] 
          # Transformar utilizando la escala a la puntuación z
        # Sugerencia: tft.scale_to_z_score
        features_dict[_transformed_name(feature)] = tft.scale_to_z_score(data_col)

    for feature in _VOCAB_FEATURE_KEYS:
        data_col = inputs[feature] 
        # Transformar utilizando el vocabulario disponible en la columna
        # Sugerencia: Utilizar tft.compute_and_apply_vocabulary
        features_dict[_transformed_name(feature)] = tft.compute_and_apply_vocabulary(data_col)

    for feature in _HASH_STRING_FEATURE_KEYS:
        data_col = inputs[feature] 
        # Transformar mediante el hash de cadenas en cubos
        # Sugerencia: Utilice tft.hash_strings con el parámetro hash_buckets establecido en 10
        features_dict[_transformed_name(feature)] = tft.hash_strings(data_col, hash_buckets=10)
    
    ### END CODE HERE ###  

    # No change in the label
    features_dict[_LABEL_KEY] = inputs[_LABEL_KEY]

    return features_dict

Overwriting cover_transform.py


In [45]:
# Test your preprocessing_fn

import cover_transform
from testing_values import feature_description, raw_data

# NOTA: Estas dos líneas siguientes son para recargar tu módulo cover_transform en caso de que necesites 
# actualizar su solución inicial y volver a ejecutar esta celda. Por favor, no las elimine, especialmente si usted
# ha revisado su solución. De lo contrario, sus cambios no serán detectados.
import importlib
importlib.reload(cover_transform)

raw_data_metadata = dataset_metadata.DatasetMetadata(schema_utils.schema_from_feature_spec(feature_description))

with tft_beam.Context(temp_dir=tempfile.mkdtemp()):
    transformed_dataset, _ = (
        (raw_data, raw_data_metadata) | tft_beam.AnalyzeAndTransformDataset(cover_transform.preprocessing_fn))

transformed_data, transformed_metadata = transformed_dataset

In [46]:
# Test that the transformed data matches the expected output
transformed_data

[{'Cover_Type': 4,
  'Elevation_xf': 0.0,
  'Hillshade_9am_xf': 1.0,
  'Hillshade_Noon_xf': 1.0,
  'Horizontal_Distance_To_Fire_Points_xf': 1.0,
  'Horizontal_Distance_To_Hydrology_xf': 1.0,
  'Horizontal_Distance_To_Roadways_xf': 0.0,
  'Slope_xf': 0.0,
  'Soil_Type_xf': 4,
  'Vertical_Distance_To_Hydrology_xf': 0.5,
  'Wilderness_Area_xf': 0}]

**Expected Output:**

```
[{'Cover_Type': 4,
  'Elevation_xf': 0.0,
  'Hillshade_9am_xf': 1.0,
  'Hillshade_Noon_xf': 1.0,
  'Horizontal_Distance_To_Fire_Points_xf': 1.0,
  'Horizontal_Distance_To_Hydrology_xf': 1.0,
  'Horizontal_Distance_To_Roadways_xf': 0.0,
  'Slope_xf': 0.0,
  'Soil_Type_xf': 4,
  'Vertical_Distance_To_Hydrology_xf': 0.5,
  'Wilderness_Area_xf': 0}]
```

In [47]:
# Test that the transformed metadata's schema matches the expected output
MessageToDict(transformed_metadata.schema)

{'feature': [{'name': 'Cover_Type',
   'type': 'INT',
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 'Elevation_xf',
   'type': 'FLOAT',
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 'Hillshade_9am_xf',
   'type': 'FLOAT',
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 'Hillshade_Noon_xf',
   'type': 'FLOAT',
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 'Horizontal_Distance_To_Fire_Points_xf',
   'type': 'FLOAT',
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 'Horizontal_Distance_To_Hydrology_xf',
   'type': 'FLOAT',
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 'Horizontal_Distance_To_Roadways_xf',
   'type': 'FLOAT',
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 'Slope_xf',
   'type': 'FLOAT',
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 'Soil_Type_xf',
   'type': 'INT',
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 

**Expected Output:**

```
{'feature': [{'name': 'Cover_Type',
   'type': 'INT',
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 'Elevation_xf',
   'type': 'FLOAT',
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 'Hillshade_9am_xf',
   'type': 'FLOAT',
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 'Hillshade_Noon_xf',
   'type': 'FLOAT',
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 'Horizontal_Distance_To_Fire_Points_xf',
   'type': 'FLOAT',
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 'Horizontal_Distance_To_Hydrology_xf',
   'type': 'FLOAT',
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 'Horizontal_Distance_To_Roadways_xf',
   'type': 'FLOAT',
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 'Slope_xf',
   'type': 'FLOAT',
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 'Soil_Type_xf',
   'type': 'INT',
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 'Vertical_Distance_To_Hydrology_xf',
   'type': 'FLOAT',
   'presence': {'minFraction': 1.0},
   'shape': {}},
  {'name': 'Wilderness_Area_xf',
   'type': 'INT',
   'intDomain': {'isCategorical': True},
   'presence': {'minFraction': 1.0},
   'shape': {}}]}
```

<a name='ex-11'></a>
#### Exercise 11: Transform

Utilice el [componente TFX Transform](https://www.tensorflow.org/tfx/api_docs/python/tfx/components/Transform) para realizar las transformaciones y generar el gráfico de transformación. Tendrá que pasar los ejemplos del conjunto de datos, el esquema *curado* y el módulo que contiene la función de preprocesamiento.

In [50]:
### START CODE HERE ###
# Instantiate the Transform component
transform = tfx.components.Transform(
    examples    = example_gen.outputs['examples'],
    schema      = schema_gen.outputs['schema'],
    module_file = os.path.abspath(_cover_transform_module_file))
    
    
    
### END CODE HERE ###

# Run the component
context.run(transform, enable_cache=False)

ExecutionResult(
    component_id: Transform
    execution_id: 36
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 30
        type_id: 23
        uri: "./pipeline/Transform/transform_graph/36"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 23
        name: "TransformGraph"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 31
        type_id: 14
        uri: "./pipeline/Transform/transformed_examples/36"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 32
        type_id: 24
        uri: "./pipeline/Transform/updated_analyzer_cache/36"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
        , artifact_type: id: 24
        name: "TransformCache"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        pre_transform_schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 33
        type_id: 18
        uri: "./pipeline/Transform/pre_transform_schema/36"
        custom_properties {
          key: "name"
          value {
            string_value: "pre_transform_schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.3.0"
          }
        }
        state: LIVE
  

Inspeccionemos algunos ejemplos del conjunto de datos transformados para ver si las transformaciones se realizan correctamente.

In [51]:
try:
    transform_uri = transform.outputs['transformed_examples'].get()[0].uri

# for grading since context.run() does not work outside the notebook
except IndexError:
    print("context.run() was no-op")
    examples_path = './pipeline/Transform/transformed_examples'
    dir_id = os.listdir(examples_path)[0]
    transform_uri = f'{examples_path}/{dir_id}'

In [52]:
# Get the URI of the output artifact representing the transformed examples
train_uri = os.path.join(transform_uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
transformed_dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

In [53]:
# import helper function to get examples from the dataset
from util import get_records

# Get 3 records from the dataset
sample_records_xf = get_records(transformed_dataset, 3)

# Print the output
pp.pprint(sample_records_xf)

[{'features': {'feature': {'Cover_Type': {'int64List': {'value': ['4']}},
                           'Elevation_xf': {'floatList': {'value': [-1.2982628]}},
                           'Hillshade_9am_xf': {'floatList': {'value': [0.87007874]}},
                           'Hillshade_Noon_xf': {'floatList': {'value': [0.9133858]}},
                           'Horizontal_Distance_To_Fire_Points_xf': {'floatList': {'value': [0.875366]}},
                           'Horizontal_Distance_To_Hydrology_xf': {'floatList': {'value': [0.18468146]}},
                           'Horizontal_Distance_To_Roadways_xf': {'floatList': {'value': [-1.1803539]}},
                           'Slope_xf': {'floatList': {'value': [-1.483387]}},
                           'Soil_Type_xf': {'int64List': {'value': ['4']}},
                           'Vertical_Distance_To_Hydrology_xf': {'floatList': {'value': [0.22351421]}},
                           'Wilderness_Area_xf': {'int64List': {'value': ['0']}}}}},
 {'featur

<a name='5'></a>
## 5 - ML Metadata

TFX utiliza [ML Metadata](https://www.tensorflow.org/tfx/guide/mlmd) bajo el capó para mantener registros de los artefactos que utiliza cada componente. Esto facilita el seguimiento de cómo se ejecuta la tubería para que pueda solucionar problemas si es necesario o quiere reproducir los resultados.

En esta sección final de la tarea, demostrarás cómo recorrer este almacén de metadatos para recuperar artefactos relacionados. Esta habilidad es útil para cuando quiera recordar qué entradas se alimentan a una etapa particular de la tubería. Por ejemplo, puede saber dónde localizar el esquema utilizado para realizar la transformación de características, o puede determinar qué conjunto de ejemplos se utilizó para entrenar un modelo.

Empezarás importando los módulos relevantes y configurando la conexión con el almacén de metadatos. También hemos proporcionado algunas funciones de ayuda para mostrar la información de los artefactos y puedes revisar su código en el módulo externo `util.py` en tu espacio de trabajo del laboratorio.

In [54]:
# Import mlmd and utilities
import ml_metadata as mlmd
from ml_metadata.proto import metadata_store_pb2
from util import display_types, display_artifacts, display_properties

# Get the connection config to connect to the metadata store
connection_config = context.metadata_connection_config

# Instantiate a MetadataStore instance with the connection config
store = mlmd.MetadataStore(connection_config)

# Declare the base directory where All TFX artifacts are stored
base_dir = connection_config.sqlite.filename_uri.split('metadata.sqlite')[0]

<a name='5-1'></a>
#### 5.1 - Acceso a los artefactos almacenados

Con la configuración de la conexión, ahora puedes interactuar con el almacén de metadatos. Por ejemplo, puedes recuperar todos los tipos de artefactos almacenados con la función `get_artifact_types()`. Como referencia, la API está documentada [aquí](https://www.tensorflow.org/tfx/ml_metadata/api_docs/python/mlmd/MetadataStore).

In [55]:
# Get the artifact types
types = store.get_artifact_types()

# Display the results
display_types(types)

,id,name
0,14,Examples
1,16,ExampleStatistics
2,18,Schema
3,21,ExampleAnomalies
4,23,TransformGraph
5,24,TransformCache


También puede obtener una lista de artefactos para un tipo particular para ver si hay variaciones utilizadas en la tubería. Por ejemplo, usted curó un esquema en una parte anterior de la asignación, por lo que éste debería aparecer en los registros. La ejecución de la celda siguiente debería mostrar al menos dos filas: una para el esquema inferido y otra para el esquema actualizado. Si ejecutó este cuaderno antes, entonces podría ver más filas debido a los diferentes artefactos del esquema guardados en el directorio `./SchemaGen/schema`.

In [56]:
# Retrieve the transform graph list
schema_list = store.get_artifacts_by_type('Schema')

# Display artifact properties from the results
display_artifacts(store, schema_list, base_dir)

,artifact id,type,uri
0,5,Schema,./SchemaGen/schema/6
1,6,Schema,./SchemaGen/schema/7
2,7,Schema,./ImportSchemaGen/schema/8
3,8,Schema,./ImportSchemaGen/schema/9
4,14,Schema,./SchemaGen/schema/17
5,17,Schema,./SchemaGen/schema/21
6,18,Schema,./ImportSchemaGen/schema/22
7,24,Schema,./SchemaGen/schema/29
8,25,Schema,./ImportSchemaGen/schema/30
9,33,Schema,./Transform/pre_transform_schema/36


Además, también puede obtener las propiedades de un artefacto en particular. TFX declara algunas propiedades automáticamente para cada uno de sus componentes. Lo más probable es que vea nombre, estado y productor_componente para cada tipo de artefacto. Se añaden propiedades adicionales cuando es necesario. Por ejemplo, se añade una propiedad split_names en los artefactos ExampleStatistics para indicar para qué divisiones se generan las estadísticas.

In [57]:
# Get the latest TransformGraph artifact
statistics_artifact = store.get_artifacts_by_type('ExampleStatistics')[-1]

# Display the properties of the retrieved artifact
display_properties(store, statistics_artifact)

,property,value
0,name,post_transform_stats
1,state,published
2,producer_component,Transform
3,tfx_version,1.3.0


<a name='5-2'></a>
#### 5.2 - Seguimiento de artefactos

Para este ejercicio final, construirás una función que devuelva los artefactos padre de uno dado. Por ejemplo, esto debería ser capaz de listar los artefactos que fueron usados para generar una instancia particular de `TransformGraph`. 

<a name='ex-12'></a>
##### Ejercicio 12: Obtener artefactos padre

Completa el siguiente código para rastrear las entradas de un artefacto en particular.

Consejos:

* Puedes encontrar útil aquí [get_events_by_artifact_ids()](https://www.tensorflow.org/tfx/ml_metadata/api_docs/python/mlmd/MetadataStore#get_events_by_artifact_ids) y [get_events_by_execution_ids()](https://www.tensorflow.org/tfx/ml_metadata/api_docs/python/mlmd/MetadataStore#get_executions_by_id). 

* Algunos de los métodos de la clase MetadataStore (como los dos indicados anteriormente) sólo aceptan iterables, así que recuerda convertir a una lista (o conjunto) si sólo tienes un int (por ejemplo, pasa `[x]` en lugar de `x`).

In [60]:
def get_parent_artifacts(store, artifact):

    ### START CODE HERE ###
    
    # Get the artifact id of the input artifact
    artifact_id = artifact.id
    
    # Get events associated with the artifact id
    artifact_id_events = store.get_events_by_artifact_ids([artifact_id])
    
    # Desde el `artifact_id_events`, obtener los ids de ejecución de los eventos OUTPUT.
    # Convierte en un conjunto para eliminar los duplicados si los hay.
    execution_id = set( 
        event.execution_id
        for event in artifact_id_events # @REPLACE
        if event.type == metadata_store_pb2.Event.OUTPUT # @REPLACE
    )
    
    # Get the events associated with the execution_id
    execution_id_events = store.get_events_by_execution_ids(execution_id)

   # De execution_id_events, obtener los ids de artefactos de los eventos INPUT.
    # Convierte en un conjunto para eliminar los duplicados si los hay.
    parent_artifact_ids = set( 
        event.artifact_id
        for event in execution_id_events
        if event.type == metadata_store_pb2.Event.INPUT
    )
    
    #  Obtener la lista de artefactos asociados con el parent_artifact_ids
    parent_artifact_list = [artifact for artifact in store.get_artifacts_by_id(parent_artifact_ids)]

    ### END CODE HERE ###
    
    return parent_artifact_list

In [61]:
# Get an artifact instance from the metadata store
artifact_instance = store.get_artifacts_by_type('TransformGraph')[0]

# Retrieve the parent artifacts of the instance
parent_artifacts = get_parent_artifacts(store, artifact_instance)

# Display the results
display_artifacts(store, parent_artifacts, base_dir)

,artifact id,type,uri
0,22,Examples,./CsvExampleGen/examples/27
1,24,Schema,./SchemaGen/schema/29


**Expected Output:**

*Note: The ID numbers may differ.*

| artifact id | type | uri |
| ----------- | ---- | --- |
| 1	| Examples | ./CsvExampleGen/examples/1 |
| 4	| Schema | ./ImportSchemaGen/schema/4 |

**Ha completado la tarea de esta semana. Ha demostrado sus habilidades en la selección de características, la realización de una canalización de datos y la recuperación de información del almacén de metadatos. Tener la capacidad de juntar todo esto será crítico cuando se trabaje con proyectos de aprendizaje automático de grado de producción. Para la próxima semana, trabajarás con más tipos de datos y verás cómo se pueden preparar en un pipeline de ML. **¡Sigue así!**